## Populating decoding-related hex maze tables

So you've done decoding for your hex maze session and have a results xarray.
It's time to assign the decoded location to a hex so we can use it for analysis!

In [ ]:
import datajoint as dj
import numpy as np

import spyglass.common as sgc
from spyglass.common import Nwbfile
from spyglass.utils.nwb_helper_fn import get_nwb_file

from spyglass_hexmaze.hex_maze_decoding import (
    DecodedPosition,
    DecodedHexPositionSelection,
    DecodedHexPosition,
    DecodedHexPath,
)

# Make sure the session exists
nwb_file_name = "IM-1478_20220724_.nwb"

# Fetch file create date and source version to make sure it's ok
nwb_file_abspath = Nwbfile.get_abs_path(nwb_file_name)
nwbf = get_nwb_file(nwb_file_abspath)
print(f"File created on {nwbf.file_create_date[0].strftime('%d/%m/%Y %H:%M:%S')}")
print(f"Source script version {nwbf.source_script}")

/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/plugin.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-12-17 11:18:58,286][INFO]: DataJoint 0.14.6 connected to scrater@lmf-db.cin.ucsf.edu:3306


View existing entries in hex maze decode related tables

In [3]:
display(DecodedPosition())

display(DecodedHexPositionSelection())

display(DecodedHexPosition())

display(DecodedHexPath())


decoding_merge_id,nwb_file_name name of the NWB file,analysis_file_name name of the file,decoded_position_object_id
231ed383-2f43-fc37-7e28-2e1ebce17873,BraveLu20240505_.nwb,BraveLu20240505_BZ1G49CS6S.nwb,a2619f97-ec05-4e51-a833-2eeed0cdbd3d
f34661fe-39f0-9102-fc76-f337f1f98d1d,Toby20250316_.nwb,Toby20250316_NEVT5I0SVB.nwb,a49a8e82-bb49-4aa0-8551-18084563fca5
f40538a7-c27c-1672-f64d-a027bfb0205f,Toby20250316_.nwb,Toby20250316_2H4NJ1FSXL.nwb,f6ff623c-d181-41d5-93ba-86f4df483bed
fb231218-5693-1d21-6fcd-74d35ea7eefe,IM-1478_20220726_.nwb,IM-1478_20220726_8P96ZLDUTU.nwb,33cb5f8f-a93d-46e6-abff-402605c50e81


decoding_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based)
f34661fe-39f0-9102-fc76-f337f1f98d1d,Toby20250316_.nwb,1
f34661fe-39f0-9102-fc76-f337f1f98d1d,Toby20250316_.nwb,3
f34661fe-39f0-9102-fc76-f337f1f98d1d,Toby20250316_.nwb,5
f34661fe-39f0-9102-fc76-f337f1f98d1d,Toby20250316_.nwb,7
f40538a7-c27c-1672-f64d-a027bfb0205f,Toby20250316_.nwb,1
f40538a7-c27c-1672-f64d-a027bfb0205f,Toby20250316_.nwb,3
f40538a7-c27c-1672-f64d-a027bfb0205f,Toby20250316_.nwb,5
f40538a7-c27c-1672-f64d-a027bfb0205f,Toby20250316_.nwb,7
fb231218-5693-1d21-6fcd-74d35ea7eefe,IM-1478_20220726_.nwb,0


decoding_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),analysis_file_name name of the file,hex_assignment_object_id
f34661fe-39f0-9102-fc76-f337f1f98d1d,Toby20250316_.nwb,1,Toby20250316_RJ9WVVPMDO.nwb,b3bf5934-d068-4987-848d-40b4b8754e1f
f34661fe-39f0-9102-fc76-f337f1f98d1d,Toby20250316_.nwb,3,Toby20250316_6I3CC4MZEM.nwb,c52b70da-cce1-4507-bf7a-e4782ac1cf09
f34661fe-39f0-9102-fc76-f337f1f98d1d,Toby20250316_.nwb,5,Toby20250316_NJ79QQO1V8.nwb,08255fe3-81db-422c-91ab-81c586c9befb
f34661fe-39f0-9102-fc76-f337f1f98d1d,Toby20250316_.nwb,7,Toby20250316_RKHGE2C02Y.nwb,2078e45a-adec-4324-8037-80eed9dc4dc8
f40538a7-c27c-1672-f64d-a027bfb0205f,Toby20250316_.nwb,1,Toby20250316_JH6XQL7AYF.nwb,ba657538-0afc-410d-94f3-0fe6f47e70ad
f40538a7-c27c-1672-f64d-a027bfb0205f,Toby20250316_.nwb,3,Toby20250316_7LENES8XRN.nwb,4ad7e56c-cb0b-4e41-b15c-8ddeda51c245
f40538a7-c27c-1672-f64d-a027bfb0205f,Toby20250316_.nwb,5,Toby20250316_B3L9TAQH0N.nwb,4d6bea62-0694-4bed-bac0-7006a638e475
f40538a7-c27c-1672-f64d-a027bfb0205f,Toby20250316_.nwb,7,Toby20250316_W75WRGAGFN.nwb,564fd994-b2f4-4f30-b7cc-14029411064a
fb231218-5693-1d21-6fcd-74d35ea7eefe,IM-1478_20220726_.nwb,0,IM-1478_20220726_A78DJ305N6.nwb,c5ff0cdd-ea8d-48ac-accd-26debf3d68ee


decoding_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),analysis_file_name name of the file,hex_path_object_id
f34661fe-39f0-9102-fc76-f337f1f98d1d,Toby20250316_.nwb,3,Toby20250316_ADF7JDGPAU.nwb,488c1cf5-c408-4006-b3d3-3b98aa0f1caa


Grab a `merge_id` that points to the `DecodingOutput` entry you want to use

In [5]:
from spyglass.decoding.decoding_merge import DecodingOutput

decode_key = {"nwb_file_name": nwb_file_name}

print("all nwbs in decoded output:")
display(DecodingOutput.merge_get_part(decode_key, multi_source=True))

# By using multi_source=True, this returns a list. So we iterate over them
all_decodes_for_this_nwb = DecodingOutput.merge_get_part(decode_key, multi_source=True)
# For now just grab the first one
decode_output = all_decodes_for_this_nwb[0]

# Fetch results to make sure they exist
merge_id = decode_output.fetch1("KEY")
results = DecodingOutput.fetch_results(merge_id)

display(results)

print(f"Merge id: {merge_id}")

all nwbs in decoded output:


[*merge_id      nwb_file_name  unit_filter_pa sorted_spikes_ position_group decoding_param encoding_inter decoding_inter estimate_decod
 +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+
 2ab779bc-2c30- IM-1478_202207 default_exclus sorted_spikes_ sorted_spikes_ contfrag_sorte 00_r1          epoch0_block1  1             
  (Total: 1)]

[2025-12-16 12:48:19,275][WARNING]: Skipped checksum for file with hash: 4c801d15-cf27-565f-ad53-97389931f9ba, and path: /stelmo/nwb/analysis/IM-1478_20220720/IM-1478_20220720_9397c5e2-6e43-4fa2-bbfa-1a6e11908358.nc
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/xarray/namedarray/core.py:496: UserWarning: Duplicate dimension names present: dimensions {'states'} appear more than once in dims=('states', 'states'). We do not yet support duplicate dimension names, but we do allow initial construction of the object. We recommend you rename the dims immediately to become distinct, as most xarray functionality is likely to fail silently if you do not. To rename the dimensions you will need to set the ``.dims`` attribute of each variable, ``e.g. var.dims=('x0', 'x1')``.
  warnings.warn(
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/xarray/namedarray/core.py:496: UserWarning: Duplicate dimension names present: dimensions {'states'} appear more than onc

<xarray.Dataset> Size: 171GB
Dimensions:                      (time: 3384862, state_bins: 12600, states: 2,
                                  state_ind: 12600, dim_0: 12600)
Coordinates:
  * time                         (time) float64 27MB 19.54 19.55 ... 6.789e+03
  * state_ind                    (state_ind) int32 50kB 0 0 0 0 0 ... 1 1 1 1 1
  * states                       (states) object 16B 'Continuous' 'Fragmented'
    environments                 (states) object 16B ...
    encoding_groups              (states) int32 8B ...
  * state_bins                   (state_bins) object 101kB MultiIndex
  * state                        (state_bins) object 101kB 'Continuous' ... '...
  * x_position                   (state_bins) float64 101kB 14.88 ... 179.1
  * y_position                   (state_bins) float64 101kB 4.21 6.197 ... 151.2
Dimensions without coordinates: dim_0
Data variables:
    acausal_posterior            (time, state_bins) float32 171GB ...
    acausal_state_probabilities  (time, states) float32 27MB ...
    initial_conditions           (dim_0) float64 101kB ...
    discrete_state_transitions   (states, states) float64 32B ...
Attributes:
    marginal_log_likelihoods:  [-2058241.  -2051544.  -2048180.2 -2046308.5 -...

Merge id: {'merge_id': UUID('2ab779bc-2c30-5b6a-a343-d8f9e59aae17')}


Populate `DecodedPosition`

This gets the most likely decoded x and y position at each time point.
Populating a single entry may take a long time, depending on how long your decoding interval is (30+ mins on breeze)

In [6]:
from spyglass_hexmaze.hex_maze_decoding import DecodedPosition

# Create a key with the merge_id from DecodingOutput and the nwb_file_name
decoded_pos_key = {
    "decoding_merge_id": str(merge_id["merge_id"]),
    "nwb_file_name": nwb_file_name,
    "epoch": 0,
}
print(decoded_pos_key)

# Populate DecodedPosition
DecodedPosition.populate(decoded_pos_key)

{'decoding_merge_id': '2ab779bc-2c30-5b6a-a343-d8f9e59aae17', 'nwb_file_name': 'IM-1478_20220720_.nwb', 'epoch': 0}


[2025-12-16 12:48:35,295][WARNING]: Skipped checksum for file with hash: 4c801d15-cf27-565f-ad53-97389931f9ba, and path: /stelmo/nwb/analysis/IM-1478_20220720/IM-1478_20220720_9397c5e2-6e43-4fa2-bbfa-1a6e11908358.nc
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/xarray/namedarray/core.py:496: UserWarning: Duplicate dimension names present: dimensions {'states'} appear more than once in dims=('states', 'states'). We do not yet support duplicate dimension names, but we do allow initial construction of the object. We recommend you rename the dims immediately to become distinct, as most xarray functionality is likely to fail silently if you do not. To rename the dimensions you will need to set the ``.dims`` attribute of each variable, ``e.g. var.dims=('x0', 'x1')``.
  warnings.warn(
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/xarray/namedarray/core.py:496: UserWarning: Duplicate dimension names present: dimensions {'states'} appear more than onc

{'success_count': 1, 'error_list': []}

Make sure it worked!

In [7]:
# Show our newly populated entry in the table
display(DecodedPosition() & decoded_pos_key)

# Fetch the df of max likelihood x,y decoded position
decoded_pos_df = (DecodedPosition & decoded_pos_key).fetch1_dataframe()
display(decoded_pos_df)

decoding_merge_id,nwb_file_name name of the NWB file,analysis_file_name name of the file,decoded_position_object_id
2ab779bc-2c30-5b6a-a343-d8f9e59aae17,IM-1478_20220720_.nwb,IM-1478_20220720_UYI4L03RTH.nwb,d83c217d-eab7-417c-9581-116b85678ac8


,hpd_thresh,spatial_cov,pred_x,pred_y
time,,,,
19.544026,0.014037,14,92.059067,49.903220
19.546026,0.010040,18,90.080151,47.916552
19.548026,0.010085,21,90.080151,47.916552
19.550026,0.008769,24,90.080151,47.916552
19.552026,0.007693,26,90.080151,47.916552
...,...,...,...,...
6789.257907,0.000373,1773,92.059067,71.756562
6789.259907,0.000373,1773,92.059067,71.756562
6789.261907,0.000373,1773,92.059067,71.756562


### Now assign this decoded position to a hex.

In [8]:
from spyglass_hexmaze.hex_maze_decoding import (
    DecodedHexPositionSelection,
    DecodedHexPosition,
)

# Insert into selection table!
DecodedHexPositionSelection.insert1(decoded_pos_key, skip_duplicates=True)

# Make sure it worked
display(DecodedHexPositionSelection() & decoded_pos_key)

decoding_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based)
2ab779bc-2c30-5b6a-a343-d8f9e59aae17,IM-1478_20220720_.nwb,0


In [11]:
# Run populate to assign to hex!
DecodedHexPosition.populate(decoded_pos_key)
DecodedHexPath.populate(decoded_pos_key)

# Make sure it worked
display(DecodedHexPosition() & decoded_pos_key)
display(DecodedHexPath() & decoded_pos_key)

[13:51:31][INFO] Spyglass: Writing new NWB file IM-1478_20220720_GIRZITV94S.nwb
INFO:spyglass:Writing new NWB file IM-1478_20220720_GIRZITV94S.nwb


decoding_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),analysis_file_name name of the file,hex_assignment_object_id
2ab779bc-2c30-5b6a-a343-d8f9e59aae17,IM-1478_20220720_.nwb,0,IM-1478_20220720_1MLS6C5W8K.nwb,8be981cd-c5c4-45ab-8f87-720185ba5102


## Fetching data

In [10]:
# Fetch the df of decode assigned to hex
decoded_hex_df = (DecodedHexPosition & decoded_pos_key).fetch1_dataframe()
display(decoded_hex_df)

,hex,hex_including_sides,distance_from_centroid
time,,,
19.544026,8,8,2.649029
19.546026,8,8,3.553444
19.548026,8,8,3.553444
19.550026,8,8,3.553444
19.552026,8,8,3.553444
...,...,...,...
6789.257907,17,17,0.613474
6789.259907,17,17,0.613474
6789.261907,17,17,0.613474


In [11]:
# Fetch the combined hex and decoded position dataframe
combined_df = (DecodedHexPosition & decoded_pos_key).fetch_hex_and_position_dataframe()
display(combined_df)

,hpd_thresh,spatial_cov,pred_x,pred_y,hex,hex_including_sides,distance_from_centroid
time,,,,,,,
49.924743,0.000059,1226,123.824107,50.677172,9,9,4.717731
49.926743,0.000059,797,123.824107,50.677172,9,9,4.717731
49.928743,0.000045,225,125.811777,52.662603,9,9,2.455686
49.930743,0.000061,99,125.811777,52.662603,9,9,2.455686
49.932743,0.001249,27,125.811777,52.662603,9,9,2.455686
...,...,...,...,...,...,...,...
1663.932692,0.000166,1694,52.267987,138.036134,49,49_left,2.277534
1663.934692,0.000154,1622,52.267987,138.036134,49,49_left,2.277534
1663.936692,0.000158,1728,54.255657,138.036134,49,49_left,3.617987


## Below this is just other maybe helpful stuff but also you can ignore it.

---------------------------------------------
 Merge keys are hard sometimes. 
 
 I have a helper method `get_all_valid_keys` that finds all valid keys to insert into `DecodedHexPositionSelection`

Valid means the session exists in `HexMazeBlock`, `DecodedPosition`, and `HexCentroids`.

In [53]:
from spyglass_hexmaze.hex_maze_decoding import (
    DecodedHexPositionSelection,
    DecodedHexPosition,
)

# Get all valid keys for the DecodedHexPositionSelection table for this nwb
# (valid = the session has HexMazeBlock, DecodedPosition, and HexCentroids data)
all_valid_keys = DecodedHexPositionSelection.get_all_valid_keys(verbose=False)
nwb_file_keys = [key for key in all_valid_keys if key["nwb_file_name"] == nwb_file_name]

if not nwb_file_keys:
    print(f"No valid HexPositionSelection keys found for {nwb_file_name}")

# Insert each key into HexPositionSelection
for key in nwb_file_keys:

    # Skip inserting the key if it already exists in the table
    if key in DecodedHexPositionSelection:
        continue
    try:
        DecodedHexPositionSelection.insert1(key, skip_duplicates=True)
        print(f"Inserted new key {key} into DecodedHexPositionSelection")
    except Exception as e:
        print(f"Skipping insert for {key}: {e}")

In [54]:
# Only populate HexPosition with keys for this nwb
selection_keys = (DecodedHexPositionSelection & {"nwb_file_name": nwb_file_name}).fetch(
    "KEY"
)
print(selection_keys)
print(f"Populating HexPosition for {len(selection_keys)} entries in {nwb_file_name}")
DecodedHexPosition.populate(selection_keys)

[{'decoding_merge_id': UUID('fb231218-5693-1d21-6fcd-74d35ea7eefe'), 'nwb_file_name': 'IM-1478_20220726_.nwb', 'epoch': 0}]
Populating HexPosition for 1 entries in IM-1478_20220726_.nwb


{'success_count': 0, 'error_list': []}